# Custom Agent Executors (Non-Streaming)

## Overview

This notebook demonstrates how to create **custom executor classes** that wrap AI agents for workflow integration. This is similar to Step 3 from the start-here folder but focuses on the custom executor pattern.

### Workflow:
```
Writer (custom executor) → Reviewer (custom executor)
```

### Key Concepts:
- **Custom Executor Classes**: Wrapping `ChatAgent` in `Executor` subclasses
- **`@handler` Pattern**: Defining workflow behavior with handler methods
- **Typed Contexts**: Using `WorkflowContext[T_Out, T_W_Out]` for type safety
- **Non-Streaming Execution**: Using `workflow.run()` for complete responses
- **Agent Lifecycle Management**: Initializing and managing agents within executors

### Prerequisites:
- ✅ Azure OpenAI configured
- ✅ Azure CLI authentication (`az login`)

In [ ]:
import asyncio

import os
from dotenv import load_dotenv
from agent_framework import (
    ChatAgent,
    ChatMessage,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
# Load environment variables from .env file
load_dotenv('../../.env')


## Define Writer Executor

### Custom Executor Pattern

This demonstrates:
- Creating a reusable executor class
- Embedding a `ChatAgent` within the executor
- Handler contract: `ChatMessage` → `list[ChatMessage]`

In [ ]:
class Writer(Executor):
    """Custom executor that owns a domain-specific agent responsible for generating content."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "writer"):
        # Create a domain-specific agent
        self.agent = chat_client.create_agent(
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
        )
        # Associate the agent with this executor node
        super().__init__(id=id)

    @handler
    async def handle(self, message: ChatMessage, ctx: WorkflowContext[list[ChatMessage], str]) -> None:
        """Generate content using the agent and forward the updated conversation."""
        # Start the conversation with the incoming user message
        messages: list[ChatMessage] = [message]
        
        # Run the agent and extend the conversation with the agent's messages
        response = await self.agent.run(messages)
        messages.extend(response.messages)
        
        # Forward the accumulated messages to the next executor in the workflow
        await ctx.send_message(messages)

print("✅ Writer executor class defined")

## Define Reviewer Executor

### Terminal Node Pattern

The Reviewer is a terminal executor that:
- Consumes the full conversation
- Yields final output via `ctx.yield_output()`

In [ ]:
class Reviewer(Executor):
    """Custom executor that owns a review agent and completes the workflow."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "reviewer"):
        # Create a domain-specific agent that evaluates and refines content
        self.agent = chat_client.create_agent(
            instructions=(
                "You are an excellent content reviewer. You review the content and provide feedback to the writer."
            ),
        )
        super().__init__(id=id)

    @handler
    async def handle(self, messages: list[ChatMessage], ctx: WorkflowContext[list[ChatMessage], str]) -> None:
        """Review the full conversation transcript and complete with a final string."""
        response = await self.agent.run(messages)
        await ctx.yield_output(response.text)

print("✅ Reviewer executor class defined")

In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
print("✅ Azure OpenAI Chat Client created")

In [ ]:
# Instantiate the two agent-backed executors
writer = Writer(chat_client)
reviewer = Reviewer(chat_client)
print("✅ Writer and Reviewer instances created")

In [ ]:
# Build the workflow
workflow = (
    WorkflowBuilder()
    .set_start_executor(writer)
    .add_edge(writer, reviewer)
    .build()
)
print("✅ Workflow built: Writer → Reviewer")

In [ ]:
# Run the workflow (non-streaming)
user_message = "Create a slogan for a new electric SUV that is affordable and fun to drive."
print(f"\n📝 User: {user_message}\n" + "=" * 70)

events = await workflow.run(
    ChatMessage(role="user", text=user_message)
)

# Get and print the output
outputs = events.get_outputs()
if outputs:
    print(f"\n📤 Final Output:\n\n{outputs[-1]}")

print("\n" + "=" * 70 + "\n✅ Complete!")

## Key Takeaways

### Custom Executor Benefits

✅ **Encapsulation**: Agent logic contained within executor class
✅ **Reusability**: Executor instances can be used across workflows
✅ **State Management**: Can maintain state across handler invocations
✅ **Lifecycle Hooks**: Can override initialization and cleanup methods

### When to Use Custom Executors vs `add_agent()`

**Use Custom Executors When:**
- Need complex initialization logic
- Require state management across invocations
- Want to add validation or preprocessing
- Building reusable workflow components

**Use `add_agent()` When:**
- Simple agent integration
- No custom state needed
- Quick prototyping
- Standard agent behavior is sufficient